In [ ]:
import requests
from cleantext import clean
from nltk.corpus import stopwords
import pandas as pd
import nltk
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time

In [ ]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

main_url = 'https://www.ccu.edu/undergrad/'

browser.visit(main_url)

In [ ]:
html = browser.html

In [ ]:
program_soup = bs(html, 'lxml')

In [ ]:
program_urls = program_soup.select('li a.uk-panel')

In [ ]:
webpage_info = []

for u in program_urls:
    page_dict = {}
    program_link = u.get('href')
    program_url_full = f'https://www.ccu.edu/undergrad/{program_link}'
    browser.visit(program_url_full)
    page_html = browser.html
    page_soup = bs(page_html, 'html.parser')
    page_title = page_soup.find('title').text
    page_h1 = page_soup.find('h1').text
    page_paras = page_soup.select('div.expandable--content')
    page_text = []
    for p in page_paras:
        page_text.append(p.text)
        clean_paras = clean(page_text, no_punct=True)
        tokens = [t for t in clean_paras.split()]
        clean_tokens = tokens[:]
        for token in tokens:
            if token in stopwords.words('english'):
                clean_tokens.remove(token)
        freq = nltk.FreqDist(clean_tokens)
        freq_words = []
        for key,val in freq.most_common(10):
            words_dict = {}
            words_dict["word"] = key
            words_dict["count"] = val
            freq_words.append(words_dict) 
    print(freq_words)
    
#     page_dict['page_title'] = page_title
#     page_dict['page_h1'] = page_h1
#     page_dict['page_paras'] = 

#     webpage_info.append(page_dict)
    

In [ ]:
freq = nltk.FreqDist(clean_tokens)

In [ ]:
freq.most_common(10)

In [ ]:
freq_words = []
for key,val in freq.most_common(10):
    words_dict = {}
    words_dict["word"] = key
    words_dict["count"] = val
    freq_words.append(words_dict)

In [ ]:
freq_words

In [ ]:
clean_tokens = tokens[:]

In [ ]:
for token in tokens:
    if token in stopwords.words('english'):
        clean_tokens.remove(token)

In [ ]:
clean_tokens

In [ ]:
for p in page_text:
    clean(page_text, no_punct=True)
    print(p)

In [ ]:
page_info = {}

In [ ]:
page_info["page_title"] = page_title
page_info["page_h1"] = page_h1
page_info["page_text"] = page_text

In [ ]:
page_info

In [ ]:
from cleantext import clean

In [ ]:
p_text = clean(p.text, no_punct=True)

In [ ]:
p_text

In [ ]:
tokens = [t for t in p_text.split()]

In [ ]:
token_text = []

tokens

In [ ]:
clean_page_text = []

for p in page_text:
    p_text = clean(p.text, no_punct=True)
    tokens = [t for t in p_text.split()]
    clean_page_text.append(tokens)

In [ ]:
browser.quit()